# Switching Backends

One use case for Ibis's portable API is the ability to develop a query locally
(using `duckdb`, for example) on a subset of the data, then rerun that same
query on the full dataset (using `bigquery`, for example) without rewriting
your code.

In this notebook we'll develop some queries locally on a subset of a real world
dataset, then rerun those same queries on the full remote dataset.

## IMDB Dataset

For this section, we'll use some of Ibis' built-in example datasets,
specifically, some IMDB data.

**Note**: the full data for both of these tables is available in
`ibis.examples.imdb_title_ratings` and `ibis.examples.imdb_title_basics`, but
we're not using those in-person to avoid everyone downloading the same 250mb
file at once.

In [ ]:
from pathlib import Path

filenames = [
    "imdb_title_basics_sample_5.parquet",
    "imdb_title_ratings.parquet",
]

folder = Path("imdb_smol")
folder.mkdir(exist_ok=True)

In [ ]:
for filename in filenames:
    path = folder / filename
    if not path.exists():
        import urllib.request

        urllib.request.urlretrieve(
            f"https://storage.googleapis.com/ibis-tutorial-data/imdb/2024-03-22/{filename}",
            path,
        )

In [ ]:
!ls imdb_smol/

### Parquet loading

In the previous examples we used a pre-existing DuckDB database, and some
in-memory tables. Another common pattern is that you have a few parquet files
you want to work with. We can load those in to an in-memory DuckDB connection.
(Note that "in-memory" here just means ephemeral, DuckDB is still very happy to
operate on as much data as your hard drive can hold)

In [ ]:
import ibis
from ibis import _

ibis.options.interactive = True

In [ ]:
con = ibis.duckdb.connect()

In [ ]:
basics = con.read_parquet(
    "imdb_smol/imdb_title_basics_sample_5.parquet", table_name="imdb_title_basics"
)

In [ ]:
ratings = con.read_parquet(
    "imdb_smol/imdb_title_ratings.parquet", table_name="imdb_title_ratings"
)

The `read_parquet` method returns an Ibis table that points to the
to-be-ingested `parquet` file. 

`read_parquet` also registers the table with DuckDB (or another backend), so
you can also load the tables like we did for the `penguins` table in the
previous notebook.

In [ ]:
basics = con.tables.imdb_title_basics  # this cell is redundant, just here for demonstration

In [ ]:
ratings = con.tables.imdb_title_ratings  # this cell is redundant, just here for demonstration

In [ ]:
basics

## Exercises

### Exercise 1

Join `basics` with `ratings` on the `tconst` column.


#### Solution

In [ ]:
%load solutions/nb03_ex01.py

### Exercise 2

Join `basics` with `ratings` on `tconst`, and select out only the `titleType`,
`primaryTitle`, `numVotes`, `averageRating`, and `isAdult`  columns.


#### Solution

In [ ]:
%load solutions/nb03_ex02.py

### Exercise 3

Those `camelCase` column names aren't [PEP
8](https://peps.python.org/pep-0008/) compliant, and feel a bit clunky to use.
Modify the above to change them to `snake_case` (for example, rename
`titleType` to `title_type`).

There are two ways you might achieve this:

- Using the `Table.rename` method
- Or by modifying the `.select` used above to do the relabeling in one step.


#### Solution

In [ ]:
%load solutions/nb03_ex03_rename.py

In [ ]:
%load solutions/nb03_ex03_select.py

### Exercise 4

Using the above joined table, compute the 10 non-adult movies with the highest
average rating having received at least 100,000 votes.


#### Solution

In [ ]:
%load solutions/nb03_ex04.py

## Expression portability

Let's turn off interactive mode for a moment.

In [ ]:
ibis.options.interactive = False

We've built up an Ibis expression that returns the top 10 highest rated films on
IMDB for certain criteria.  The data we used is only a 5% sample, but the query
should work no matter what size the data are.

Now we're going to call a special `unbind` method.

In [ ]:
expr = sol4.unbind()

What does `unbind` do?  It takes an Ibis expression that was created using one
backend (in our case, `DuckDB`) and generalizes it into an "unbound" expression.

Let's look at the `repr` of the unbound expression (this will error if you have
interactive mode on, that's why we turned it off).

In [ ]:
expr

The only visible difference is that we see "UnboundTable" a bunch of times, and
that's the point.  The _expression_ is independent of the backend that created
it.

So, if we had a _different_ backend connection, that had the full IMDB ratings
data loaded into it, could we run our locally developed query against the
"production" data?

(With caveats that the table names and schema need to match).

Let's find out!

In [ ]:
pg_con = ibis.postgres.connect()

In [ ]:
pg_con.list_tables()

That seems promising.

Now, we can't call `expr.to_pandas()` because the expression isn't _bound_ to a
backend.  Ibis doesn't know (anymore) where to execute it. Instead, we can use
the `to_pandas` (or similar) method on the backend connection object itself:

In [ ]:
pg_con.to_pandas(expr, limit=10)